# HW1 Problem 2ab

This problem is a **set covering problem**.

Define the set of nodes
$N = \{ i \mid i \in \{1,2,3,4,5,6\} \}$
that must be covered. Introduce a binary decision variable $$x_i$$ such that
$$
x_i =
\begin{cases}
1, & \text{if node } i \text{ is selected}, \\
0, & \text{otherwise},
\end{cases}
\qquad \forall i \in N.
$$

Let:
- $C = \{ c_i \mid i \in N \}$ denote the cost of selecting node $i$,
- $R = \{ (1,2), (1,4), (4,5), (2,5), (2,3), (3,5), (3,6), (5,6) \}$ denote the set of road links.

### Integer Linear Programming Formulation

**Objective:**
$$
\min \sum_{i \in N} x_i c_i
$$

**Subject to:**
$$
\begin{aligned}
x_1 + x_2 &\ge 1 \quad &\text{(link 1--2)} \\
x_1 + x_4 &\ge 1 \quad &\text{(link 1--4)} \\
x_4 + x_5 &\ge 1 \quad &\text{(link 4--5)} \\
x_2 + x_5 &\ge 1 \quad &\text{(link 2--5)} \\
x_2 + x_3 &\ge 1 \quad &\text{(link 2--3)} \\
x_3 + x_5 &\ge 1 \quad &\text{(link 3--5)} \\
x_3 + x_6 &\ge 1 \quad &\text{(link 3--6)} \\
x_5 + x_6 &\ge 1 \quad &\text{(link 5--6)}
\end{aligned}
$$

**Binary constraints:**
$$
x_i \in \{0,1\} \quad \forall i \in N
$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
# Set of nodes
N = set(range(1, 7))

# Road links
R = {
    (1, 2): 1,
    (1, 4): 1,
    (4, 5): 1,
    (2, 5): 1,
    (2, 3): 1,
    (5, 3): 1,
    (5, 6): 1,
    (3, 6): 1,
}

# Cost of selecting each node
C = {
    1: 40,
    2: 65,
    3: 43,
    4: 48,
    5: 72,
    6: 36,
}


In [3]:
# Model
m = pe.ConcreteModel()

# Sets
m.nodes = pe.Set(initialize=N)
m.routes = pe.Set(initialize=R.keys(), dimen=2)

# Parameters
m.cost = pe.Param(m.nodes, initialize=C)

# Variables
m.x = pe.Var(m.nodes, within=pe.Binary)

# Objective
m.obj = pe.Objective(
    expr=sum(m.cost[i] * m.x[i] for i in m.nodes),
    sense=pe.minimize
)

# Constraints
def cover_rule(m, i, j):
    return m.x[i] + m.x[j] >= 1

m.cover = pe.Constraint(m.routes, rule=cover_rule)


source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [4]:
opt = po.SolverFactory('gurobi', solver_io='python')
results = opt.solve(m, tee=True)

m.display()

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 6 columns and 16 nonzeros (Min)
Model fingerprint: 0x0723d698
Model has 6 linear objective coefficients
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Found heuristic solution: objective 221.0000000
Presolve removed 8 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 155 221 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.550000000000e+02, best bound 1.550000000000e+02, gap 0.0000%
Model unknown

  Variables:
    x : Size=6, Index=nod

In [5]:
chosen = [i for i in m.nodes if pe.value(m.x[i]) > 0.5]
total_cost = sum(pe.value(m.cost[i]) for i in chosen)

print("Chosen nodes:", chosen)
print("Total cost:", total_cost)

Chosen nodes: [1, 3, 5]
Total cost: 155
